Training Breathing Bag Defects

In [1]:
import tensorflow as tf 
print(tf.config.list_physical_devices('GPU'))

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [2]:
import os
import sys
import json
import datetime
import numpy as np
import skimage.draw
import cv2
import imgaug.augmenters as iaa
from mrcnn.visualize import display_instances
import matplotlib.pyplot as plt
#from mrcnn.utils import TensorBoard

#Ignoring converting error during training
import warnings
warnings.filterwarnings("ignore", message="Converting sparse IndexedSlices.*")

# Root directory of the project
ROOT_DIR = "./"

# Import Mask RCNN
sys.path.append(ROOT_DIR)  # To find local version of the library
from mrcnn.config import Config
from mrcnn import model as modellib, utils

# Path to trained weights file
# COCO_WEIGHTS_PATH = os.path.join(ROOT_DIR, "mask_rcnn_coco.h5") #Orginal
COCO_WEIGHTS_PATH = os.path.join(ROOT_DIR, "mask_rcnn_coco.h5") 

# Directory to save logs and model checkpoints, if not provided
# through the command line argument --logs
DEFAULT_LOGS_DIR = os.path.join(ROOT_DIR, "logs")

Config setup

In [3]:
class CustomConfig(Config):
    """Configuration for training on the custom  dataset.
    Derives from the base Config class and overrides some values.
    """
    # Give the configuration a recognizable name
    NAME = "object"

    # We use a GPU with 12GB memory, which can fit two images.
    # Adjust down if you use a smaller GPU.
    IMAGES_PER_GPU = 1

    # Number of classes (including background)
    NUM_CLASSES = 1 + 1  # Background + Hole  # car and truck

    RPN_ANCHOR_SCALES = (4, 8, 16, 32, 64)  # anchor side in pixels
    
    #LEARNING_RATE = 0.001
    
    # Backbone network architecture
    # Supported values are: resnet50, resnet101.
    #BACKBONE = "resnet101"
    
    DETECTION_NMS_THRESHOLD = 0.001
    
    # Number of validation steps to run at the end of every training epoch.
    # A bigger number improves accuracy of validation stats, but slows
    # down the training.
    VALIDATION_STEPS = 50
    
    USE_MINI_MASK = False # True from start, try False to calc mAP see discord

    # Number of training steps per epoch
    STEPS_PER_EPOCH = 2000

    # Skip detections with < 50% confidence
    DETECTION_MIN_CONFIDENCE = 0.5

Dataset

In [4]:
class CustomDataset(utils.Dataset):

    def load_custom(self, dataset_dir, subset):
        """Load a subset of the Dog-Cat dataset.
        dataset_dir: Root directory of the dataset.
        subset: Subset to load: train or val
        """
        # Add classes. We have only one class to add.
        self.add_class("object", 1, "Hole")
        #self.add_class("object", 2, "Discoloration")

        # Train or validation dataset?
        assert subset in ["train", "val"]
        dataset_dir = os.path.join(dataset_dir, subset)

        # We mostly care about the x and y coordinates of each region
        annotations1 = json.load(open("Dataset_manual/" + subset + "/dataset.json"))
        # print(annotations1)
        annotations = list(annotations1.values())  # don't need the dict keys

        # The VIA tool saves images in the JSON even if they don't have any
        # annotations. Skip unannotated images.
        annotations = [a for a in annotations if a['regions']]
        
        # Add images
        for a in annotations:
            # print(a)
            # Get the x, y coordinaets of points of the polygons that make up
            # the outline of each object instance. There are stores in the
            # shape_attributes (see json format above)
            polygons = [r['shape_attributes'] for r in a['regions']] 
            objects = [s['region_attributes']['names'] for s in a['regions']]
            print("objects:",objects)
            name_dict = {"Hole": 1}  # Lägg komma efteråt följt av nästa klass vid flera

            # key = tuple(name_dict)
            num_ids = [name_dict[a] for a in objects]
     
            # num_ids = [int(n['Event']) for n in objects]
            # load_mask() needs the image size to convert polygons to masks.
            # Unfortunately, VIA doesn't include it in JSON, so we must read
            # the image. This is only managable since the dataset is tiny.
            print("numids",num_ids)
            image_path = os.path.join(dataset_dir, a['filename'])
            image = skimage.io.imread(image_path)
            height, width = image.shape[:2]

            self.add_image(
                "object",  ## for a single class just add the name here
                image_id=a['filename'],  # use file name as a unique image id
                path=image_path,
                width=width, height=height,
                polygons=polygons,
                num_ids=num_ids
                )

    def load_mask(self, image_id):
        """Generate instance masks for an image.
       Returns:
        masks: A bool array of shape [height, width, instance count] with
            one mask per instance.
        class_ids: a 1D array of class IDs of the instance masks.
        """
        # If not a Dog-Cat dataset image, delegate to parent class.
        image_info = self.image_info[image_id]
        if image_info["source"] != "object":
            return super(self.__class__, self).load_mask(image_id)

        # Convert polygons to a bitmap mask of shape
        # [height, width, instance_count]
        info = self.image_info[image_id]
        if info["source"] != "object":
            return super(self.__class__, self).load_mask(image_id)
        num_ids = info['num_ids']
        mask = np.zeros([info["height"], info["width"], len(info["polygons"])],
                        dtype=np.uint8)
        for i, p in enumerate(info["polygons"]):
            # Get indexes of pixels inside the polygon and set them to 1
        	rr, cc = skimage.draw.polygon(p['all_points_y'], p['all_points_x'])

        	mask[rr, cc, i] = 1

        # Return mask, and array of class IDs of each instance. Since we have
        # one class ID only, we return an array of 1s
        # Map class names to class IDs.
        num_ids = np.array(num_ids, dtype=np.int32)
        return mask, num_ids #np.ones([mask.shape[-1]], dtype=np.int32)

    def image_reference(self, image_id):
        """Return the path of the image."""
        info = self.image_info[image_id]
        if info["source"] == "object":
            return info["path"]
        else:
            super(self.__class__, self).image_reference(image_id)


In [5]:
def train(model):
    """Train the model."""
    # Training dataset.
    dataset_train = CustomDataset()
    dataset_train.load_custom("Dataset_manual", "train")
    dataset_train.prepare()

    # Validation dataset
    dataset_val = CustomDataset()
    dataset_val.load_custom("Dataset_manual", "val")
    dataset_val.prepare()
    
    # Define the augmentation
    augmentation = iaa.Sequential([
        iaa.Fliplr(0.5), # flip horizontally with probability 0.5
        #iaa.GaussianBlur(sigma=(0, 2.0)) # apply gaussian blur with random sigma in range 0 to 5
    ])
    
    # *** This training schedule is an example. Update to your needs ***
    # Since we're using a very small dataset, and starting from
    # COCO trained weights, we don't need to train too long. Also,
    # no need to train all layers, just the heads should do it.
    print("Training network heads")
    model.train(dataset_train, dataset_val,
                learning_rate=config.LEARNING_RATE,
                epochs=30,
                layers='heads',
                )

In [6]:
config = CustomConfig()
model = modellib.MaskRCNN(mode="training", config=config,
                                  model_dir=DEFAULT_LOGS_DIR)

In [7]:
weights_path = COCO_WEIGHTS_PATH
        # Download weights file
if not os.path.exists(weights_path):
  utils.download_trained_weights(weights_path)

model.load_weights(weights_path, by_name=True, exclude=[
            "mrcnn_class_logits", "mrcnn_bbox_fc",
            "mrcnn_bbox", "mrcnn_mask"])

train(model)

objects: ['Hole']
numids [1]
objects: ['Hole']
numids [1]
objects: ['Hole']
numids [1]
objects: ['Hole', 'Hole', 'Hole']
numids [1, 1, 1]
objects: ['Hole']
numids [1]
objects: ['Hole']
numids [1]
objects: ['Hole']
numids [1]
objects: ['Hole']
numids [1]
objects: ['Hole']
numids [1]
objects: ['Hole']
numids [1]
objects: ['Hole']
numids [1]
objects: ['Hole']
numids [1]
objects: ['Hole']
numids [1]
objects: ['Hole']
numids [1]
objects: ['Hole']
numids [1]
objects: ['Hole']
numids [1]
objects: ['Hole']
numids [1]
objects: ['Hole']
numids [1]
objects: ['Hole', 'Hole']
numids [1, 1]
objects: ['Hole']
numids [1]
objects: ['Hole']
numids [1]
objects: ['Hole']
numids [1]
objects: ['Hole']
numids [1]
objects: ['Hole']
numids [1]
objects: ['Hole']
numids [1]
objects: ['Hole']
numids [1]
objects: ['Hole']
numids [1]
objects: ['Hole']
numids [1]
objects: ['Hole']
numids [1]
objects: ['Hole']
numids [1]
objects: ['Hole']
numids [1]
objects: ['Hole']
numids [1]
objects: ['Hole']
numids [1]
objects: [

KeyboardInterrupt: 